## Chapter 12: New Data Types in Julia

In this chapter we investigate how to create a new datatype, which is 1) quite easy to do in Julia and 2) can make life much easier if designed right.

### 12.1 Basics of a Composite type

The way to create a new type in julia is to create a `struct`.  The following makes a new type with two fields

In [1]:
struct Mystruct
  num::Integer
  str::String
end

We can make an "object" of that type by calling the type like a function.  It must be called with the fields as arguments in the proper order:

In [2]:
m=Mystruct(11,"hello")

Mystruct(11, "hello")

The fields of a struct can be accessed via the . syntax:

In [3]:
m.num

11

In [4]:
m.str

"hello"

In [5]:
t = (num = 11, str = "hello")

(num = 11, str = "hello")

In [6]:
fieldnames(Mystruct)

(:num, :str)

Notice that first, this is a tuple, but also

In [7]:
typeof(fieldnames(Mystruct))

Tuple{Symbol, Symbol}

The names are `Symbol`s. 

Note that the function fieldnames is called with the type not an object the type. 

In [8]:
fieldnames(m)

MethodError: MethodError: no method matching fieldnames(::Mystruct)
The function `fieldnames` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  fieldnames(!Matched::Core.TypeofBottom)
   @ Base reflection.jl:254
  fieldnames(!Matched::Type{<:Tuple})
   @ Base reflection.jl:256
  fieldnames(!Matched::UnionAll)
   @ Base reflection.jl:253
  ...


In [9]:
m.num=15

ErrorException: setfield!: immutable struct of type Mystruct cannot be changed

structs are immutable. That is, after being created, they can't be altered. If you need one that is Mutable, add the keyword `mutable` to the front. 

In [10]:
mutable struct MutableStruct
  a::Float64
  b::Integer
end

In [11]:
s = MutableStruct(1,2)

MutableStruct(1.0, 2)

In [12]:
s.a=4.5

4.5

In [13]:
s

MutableStruct(4.5, 2)

### 12.2: A Card datatype

We are going to do some simulations soon with playing cards.  We have the ranks and suits defined as follows.  The suits are the Unicode characters for the suits.   Notice that these are arrays of characters (they are in single quotes).

In [1]:
ranks = ['A','2','3','4','5','6','7','8','9','T','J','Q','K'];
suits = ['\u2660','\u2661','\u2662','\u2663']

4-element Vector{Char}:
 '♠': Unicode U+2660 (category So: Symbol, other)
 '♡': Unicode U+2661 (category So: Symbol, other)
 '♢': Unicode U+2662 (category So: Symbol, other)
 '♣': Unicode U+2663 (category So: Symbol, other)

And we will create a `Card` struct that contains two fields, a rank and a suit.  We can make both integers:

In [15]:
struct Card
  rank::Integer
  suit::Integer
end

In [16]:
c = Card(3,2)

Card(3, 2)

This is a very boring looking 3 of hearts, so we use the following which will print out a Card much nicer:

In [5]:
Base.show(io::IO, c::Card) = print(io, string(ranks[c.rank],suits[c.suit]))

We are creating a specific `show` function that is called whenever a `Card` object is printed or anytime converted to a string. 

In [18]:
c = Card(3,2)

3♡

What happens if we do

In [19]:
Card(-10,98)

BoundsError: BoundsError: attempt to access 13-element Vector{Char} at index [-10]

This is because in the `Base.show` function that you are accessing a part of the array with index -10, which is not possible. We can check for this with:

In [3]:
struct Card
  rank::Int
  suit::Int

  function Card(r::Int,s::Int)
    1 <= r <= 13 || throw(ArgumentError("The argument for rank must be between 1 and 13"))
    1 <= s <= 4 || throw(ArgumentError("The argument for suit must be between 1 and 4"))
    new(r,s)
  end
end

The error occurs because `Card` is immutable and we are trying to redefine it.  So restart the kernel and rerun this. Also, rerun the cell with the ranks an suits arrays declared as well as the `Base.show` function.

There are other things going on with this.  
- the function starting on line 5 is called a _Constructor_ because it has the same name (and capitalization) as the `struct` and is used to construct an object.  This is called whenever `Card(r,s)` is called. 
- Line 6 checks if `r` is between 1 and 13.  If not, an error is thrown.  This is shorthand for and if then statement, but is common for error checking in Julia. 
- Similarly line 7 check if `s` is between 1 and 4. 
- `new(r,s)` assigns `rank` to `r` and `suit` to `s`.  Shorthand for `rank=r; suit-s` and is common in



In [4]:
Card(-10,8)

ArgumentError: ArgumentError: The argument for suit must be between 1 and 4

Now, we are not allowed to make a card with invalid rank and suit.

Let's expand the Card type.  It would be nice to also have a way of making a Card on a single integer.  (Later we will make an entire deck from the cards from 1 to 52).  The following does this:

In [3]:
struct Card
  rank::Int
  suit::Int
  function Card(r::Int,s::Int)
    1 <= r <= 13 || throw(ArgumentError("The argument for rank must be between 1 and 13"))
    1 <= s <= 4 || throw(ArgumentError("The argument for suit must be between 1 and 4"))
    new(r,s)
  end

  function Card(i::Int)
    1<=i<=52 || throw(ArgumentError("The argument must be an integer between 1 and 52"))
    mod(i,13)==0 ? new(13,i ÷ 13) : new(i % 13, i ÷ 13 + 1)
  end
end

This shows that you can create two or more constructors for a type.  Some things with this:
* You must define a default Constructor that fills the fields in the order given. 
* The `new(r,s)` function is a way to call that default constructor.  

In [6]:
Card(45)

6♣

In [7]:
Card(73)

ArgumentError: ArgumentError: The argument must be an integer between 1 and 52

In [8]:
Card(5,8)

ArgumentError: ArgumentError: The argument for rank must be between 1 and 13

We also would like a set of cards and we're going to use an array to do this and call it a `Hand`. 

In [9]:
struct Hand
    cards::Vector{Card}
end

This creates a hand.

In [11]:
h=Hand([Card(2,3),Card(12,1),Card(10,1),Card(10,4),Card(5,2)])

Hand(Card[2♢, Q♠, T♠, T♣, 5♡])

And it's a good idea to define a `Base.show` for a `Hand`.  This just makes a string that is calling the `Card` version of `Base.show` and then joining the cards. 

In [12]:
Base.show(io::IO,h::Hand) = print(io, string("[",join(h.cards,", "),"]"))

In [13]:
h

[2♢, Q♠, T♠, T♣, 5♡]

#### 12.3: Polynomial Datatype

In this section, we look at a polynomial datatype.  We will look at creating polynomials, adding them, evaluating them and plotting this.  First, we define a polynomial with integer coefficients.  Don't run this though:

In [19]:
struct Polynomial
  coeffs::Vector{Int64}
end

This will define a polynomial with integer coefficients, but it will be a pain, if we need to define different polynomials, with different types of coefficients.  We can create a Polynomial with any number type using:

In [1]:
struct Polynomial{T <: Number}
  coeffs::Vector{T}
end

In [2]:
poly1 = Polynomial([1,2,3])

Polynomial{Int64}([1, 2, 3])

In [3]:
poly2 = Polynomial([0.5,-0.25,6])

Polynomial{Float64}([0.5, -0.25, 6.0])

In [4]:
typeof(1//2)

Rational{Int64}

which again is a terrible way to represent a polynomial, so let's create another `Base.show` for a polynomial

In [5]:
function Base.show(io::IO, p::Polynomial)
  str = ""
  for i = 1:length(p.coeffs)
      str = string(str,p.coeffs[i],"x^",i-1,i<length(p.coeffs) ? "+" : "")
  end
  print(io, str)
end

In [8]:
p = Polynomial([1,2,3])

1 x^0 + 2 x^1 + 3 x^2

The following uses `mapreduce` to construct the output.  This is mainly to show how to use this as an example:

In [7]:
Base.show(io::IO, p::Polynomial) = print(io, mapreduce(i -> "$(p.coeffs[i]) x^$(i-1)", (str, term) -> "$str + $term" , 1:length(p.coeffs)))

Since this is complicated, let's walk through it.
- `mapreduce` like `map` starts with an array.  We will use `1:length(p.coeffs)`. 
- The first argument of `mapreduce` is to apply a function to each term.  In this case, we create each term of the polynomial.  Recall that the coefficient is in `p.coeffs[i]`.  The power will be `i-1` because the array starts at 1. 
- The reduce part of this comes from the second function which is concatenation.  The first term `str` is the accumulated string and `term` is the current term of the polynomial. 

#### Create different polynomials:

In [9]:
poly1=Polynomial([1,2,3])
poly2=Polynomial([1.0,2.0,3.0])
poly3=Polynomial([2//3,3//4,5//8])
poly4=Polynomial([im,2+0im,3-2im,-im])
poly5=Polynomial([(-1)^(n+1) for n=1:6])

1 x^0 + -1 x^1 + 1 x^2 + -1 x^3 + 1 x^4 + -1 x^5

In [10]:
poly3

2//3 x^0 + 3//4 x^1 + 5//8 x^2

Evaluate a polynomial at a number using Horner's method:

In [12]:
function eval(poly::Polynomial,x::Number)
  reduce((val,i) -> x*val+p.coeffs[i],(length(p.coeffs)-1):-1:1, init=p.coeffs[end])
end

eval (generic function with 2 methods)

In [13]:
eval(poly1,3)

34

In [14]:
1+2*3+3*3^2

34

In [15]:
eval(poly4,im)

-2 + 2im

#### Adding two polynomials:

To add two polynomails, we first need to import `Base.+`:

In [18]:
import Base.+

In [19]:
function +(p1::Polynomial,p2::Polynomial)
  Polynomial(p1.coeffs+p2.coeffs)
end

+ (generic function with 198 methods)

- to add a function to `+`, we need to import `Base.+`.  This is different than `Base.show`.  
- Since a `Polynomial` has a internal type, we want to add polynomials of different types, which is why the top line is defined the way it is. 
- We are leveraging the fact that arrays of different number types are calculated correctly and can just add the coefficiens and create a new Polynomial. 

In [20]:
poly1+poly2

2.0 x^0 + 4.0 x^1 + 6.0 x^2

In [21]:
poly1+poly5

DimensionMismatch: DimensionMismatch: a has size (3,), b has size (6,), mismatch at dim 1

In [22]:
methods(+)

# 198 methods for generic function "+" from Base:
   [1] +(x::BigInt, y::BigInt)
     @ Base.GMP gmp.jl:496
   [2] +(a::BigInt, b::BigInt, c::BigInt)
     @ Base.GMP gmp.jl:536
   [3] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt)
     @ Base.GMP gmp.jl:537
   [4] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt)
     @ Base.GMP gmp.jl:538
   [5] +(x::BigInt, y::BigInt, rest::BigInt...)
     @ Base.GMP gmp.jl:678
   [6] +(c::BigInt, x::BigFloat)
     @ Base.MPFR mpfr.jl:527
   [7] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.GMP gmp.jl:544
   [8] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8})
     @ Base.GMP gmp.jl:550
   [9] +(x::Bool, z::Complex{Bool})
     @ complex.jl:308
  [10] +(x::Bool, y::Bool)
     @ bool.jl:166
  [11] +(x::Bool)
     @ bool.jl:163
  [12] +(x::Bool, z::Complex)
     @ complex.jl:315
  [13] +(x::Real, z::Complex{Bool})
     @ complex.jl:322
  [14] +(x::Bool, y::T) where T<:AbstractFloat
     @ bool.jl:173
  [15] +(dt::Dates.Date, t::Dates.Time)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:21
  [16] +(dt::Dates.Date, y::Dates.Year)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:29
  [17] +(dt::Dates.Date, z::Dates.Month)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:56
  [18] +(x::Dates.Date, y::Dates.Quarter)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:75
  [19] +(x::Dates.Date, y::Dates.Week)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:79
  [20] +(x::Dates.Date, y::Dates.Day)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:81
  [21] +(level::Base.CoreLogging.LogLevel, inc::Integer)
     @ Base.CoreLogging logging/logging.jl:132
  [22] +(x::Rational{BigInt}, y::Rational{BigInt})
     @ Base.GMP.MPQ gmp.jl:1056
  [23] +(B::BitMatrix, J::LinearAlgebra.UniformScaling)
     @ LinearAlgebra ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/LinearAlgebra/src/uniformscaling.jl:151
  [24] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/periods.jl:335
  [25] +(x::Dates.CompoundPeriod, y::Dates.Period)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/periods.jl:333
  [26] +(x::Dates.CompoundPeriod, y::Dates.TimeType)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/periods.jl:363
  [27] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight)
     @ Pkg.Resolve ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Pkg/src/Resolve/versionweights.jl:22
  [28] +(z::Complex{Bool}, x::Bool)
     @ complex.jl:309
  [29] +(z::Complex{Bool}, x::Real)
     @ complex.jl:323
  [30] +(z::Complex, x::Bool)
     @ complex.jl:316
  [31] +(::Missing, ::Missing)
     @ missing.jl:122
  [32] +(::Missing)
     @ missing.jl:101
  [33] +(x::Missing, y::Dates.AbstractTime)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:91
  [34] +(::Missing, ::Number)
     @ missing.jl:123
  [35] +(t::Dates.Time, dt::Dates.Date)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:22
  [36] +(x::Dates.Time, y::Dates.TimePeriod)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:85
  [37] +(dt::Dates.DateTime, y::Dates.Year)
     @ Dates ~/.julia/juliaup/julia-1.11.0+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/arithmeti

#### Exercise
- Create a function `-` that subtracts two polynomials
- Create a function `*` that multiplies a constant (on the left) to a polynomial.

In [23]:
import Base.*
*(x::Number, p::Polynomial) = Polynomial(x*p.coeffs)

* (generic function with 222 methods)

In [24]:
2*poly1

2 x^0 + 4 x^1 + 6 x^2

#### 12.4: Develop a Root datatype

Recall that when we developed Newton's method in Chapter 10, we weren't sure if a function didn't have a root (or Newton's method didn't find it.)  Here we will create a type called `Root` that will store all of the important information about the results of Newton's method and return a `Root` object, instead of just a number.

We're going to create a `Root` struct that stores a lot of information about the root. The fields are:

* `root`, the approximation to the root
* `x_eps`, an approximate error to the root
* `f_eps`, an estimate of the function value at the approximate root
* `num_steps`, the number of steps of Newton's method
* `converged`, a boolean on whether or not Newton's method converged. 
* `max_steps`, the maximum number of steps specified. 

In [26]:
struct Root
  root::Float64
  x_eps::Float64
  f_eps::Float64
  num_steps::Int
  converged::Bool
  max_steps::Int
end

Here's a version of Newton's method that returns a `Root` object:

In [25]:
using ForwardDiff

function newton(f::Function, x0::Real)
  local n=0
  local dx = -f(x0)/ForwardDiff.derivative(f,x0)
  while abs(dx)>1e-6 && abs(f(x0))>1e-6
    x0 += dx
    dx = -f(x0)/ForwardDiff.derivative(f,x0)
    n += 1
    if n==10  # if too many steps are taken, break out of the while loop
      return Root(x0,dx,f(x0),n,false,10)
    end
  end
  Root(x0,dx,f(x0),n,true,10)
end

newton (generic function with 1 method)

In [27]:
newton(x->x^2-2,1)

Root(1.4142135623746899, -1.5947429102833119e-12, 4.510614104447086e-12, 4, true, 10)

This isn't much of a helpful struct, because you might not remember what each parameter is. 

In [28]:
function Base.show(io::IO,r::Root)
  if(r.converged)
    str = string("The root is approximately x̂ = $(r.root)\n")
    str = string(str,"An estimate for the error is $(r.x_eps)\n")
    str = string(str,"with f(x̂) = $(r.f_eps)\n")
    str = string(str,"which took $(r.num_steps) steps")
  else
    str = string("The root was not found within $(r.max_steps) steps.\n");
    str = string(str,"Currently, the root is approximately x̂ = $(r.root) \n")
    str = string(str,"An estimate for the error is $(r.x_eps)\n")
    str = string(str,"with f(x̂) = $(r.f_eps)\n")
  end
  print(io,str)
end

Let's rerun this now and result is much clearer.

In [31]:
r = newton(x->x^2-2,1)

The root is approximately x̂ = 1.4142135623746899
An estimate for the error is -1.5947429102833119e-12
with f(x̂) = 4.510614104447086e-12
which took 4 steps

In [32]:
r.root

1.4142135623746899

And if we put in a function that doesn't converge, will tell us as well.

In [33]:
r = newton(x->x^2+1,2)

The root was not found within 10 steps.
Currently, the root is approximately x̂ = 2.4008803928468465 
An estimate for the error is -1.4086971347905715
with f(x̂) = 6.764226660756428


In [ ]:
r.root

In [ ]:
r.f_eps